### **PASO 1: Instalación de Paquetes Necesarios**
Se instalan las bibliotecas necesarias para que el chatbot funcione correctamente.

In [12]:
import sys
print(sys.version)

# Instalación de bibliotecas necesarias
%pip install transformers
%pip install sentence_transformers
%pip install hnswlib
%pip install numpy<2.0  # Especificar versión de numpy menor que 2.0
%pip install PyPDF2  # Para extraer texto de archivos PDF
%pip install python-dotenv  # Para manejo de variables de entorno desde .env
%pip install tenacity  # Para manejar reintentos en llamadas API
%pip install llama-index  # Para integración con Gemini
%pip install llama-index-llms-gemini  # Para usar Gemini como LLM
%pip install llama-index-embeddings-gemini  # Para embeddings relacionados con Gemini
%pip install tqdm  # mostrar barras de progreso


3.10.12 (main, Dec  1 2024, 18:33:11) [Clang 14.0.3 (clang-1403.0.22.14.1)]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### **PASO 2: Importar Librerías y Configurar Logging**
Se importan las librerías necesarias y se configura un sistema de logs para monitorear todo el flujo.


In [13]:
# Importar todas las librerías necesarias
import os
import json
import logging
import hnswlib
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
import numpy as np
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from tenacity import retry, wait_exponential, stop_after_attempt
from llama_index.llms.gemini import Gemini
from llama_index.core.llms import ChatMessage
import time

# Configuración de logging para imprimir todo en consola
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]  # Solo consola
)

logging.info("Librerías importadas correctamente.")  # Mensaje de log para confirmar importación

# Cargar variables de entorno desde un archivo .env
# Assuming your .env file is in 'gdrive/MyDrive/Colab Notebooks/DIA/'
load_dotenv()
logging.info("Variables de entorno cargadas desde el archivo .env.")

2024-12-01 23:09:49,148 - INFO - Librerías importadas correctamente.
2024-12-01 23:09:49,157 - INFO - Variables de entorno cargadas desde el archivo .env.


### **PASO 3: Cargar Documentos**
Este paso carga documentos desde un archivo o un directorio. Soporta formatos .txt, .json y .pdf.


In [14]:
def load_documents(source, is_directory=False):
    """
    Carga documentos desde un archivo o un directorio. Soporta .txt, .json y .pdf.
    Además, divide los archivos .txt en unidades usando un delimitador específico.
    """
    loaded_files = []

    # Verificar si la ruta existe
    if not os.path.exists(source):
        logging.error(f"La fuente '{source}' no existe.")
        raise FileNotFoundError(f"La fuente '{source}' no se encontró.")

    if is_directory:
        logging.info(f"Iniciando carga desde el directorio: {source}.")
        for filename in os.listdir(source):
            filepath = os.path.join(source, filename)
            if os.path.isfile(filepath) and filepath.endswith(('.txt', '.json', '.pdf')):
                content = extract_content(filepath)
                if content:
                    loaded_files.append({"filename": filename, "content": content})
                    logging.info(f"Archivo '{filename}' cargado correctamente.")
    else:
        logging.info(f"Iniciando carga del archivo: {source}.")
        content = extract_content(source)
        if content:
            loaded_files.append({"filename": os.path.basename(source), "content": content})
            logging.info(f"Archivo '{os.path.basename(source)}' cargado correctamente.")

    logging.info(f"{len(loaded_files)} documentos cargados.")
    return loaded_files

def extract_content(filepath):
    """
    Extrae el contenido del archivo según su tipo (.txt, .json, .pdf).
    Si el archivo es .txt, lo divide en unidades por el delimitador '\n------\n'.
    """
    try:
        if filepath.endswith('.txt'):
            with open(filepath, 'r', encoding='utf-8') as file:
                content = file.read()
            # Dividir el contenido en unidades por el delimitador
            units = content.split("\n-----\n")
            return units  # Devolver una lista de unidades
        elif filepath.endswith('.json'):
            with open(filepath, 'r', encoding='utf-8') as file:
                return json.load(file)
        elif filepath.endswith('.pdf'):
            reader = PdfReader(filepath)
            return ''.join(page.extract_text() or '' for page in reader.pages)
    except Exception as e:
        logging.error(f"Error al extraer contenido de '{filepath}': {e}")
        return None

# Cargar documentos (True: busca en directorio, False: busca archivo)
ruta_fuente = 'data' # Usando un directorio como ejemplo
documentos = load_documents(ruta_fuente, is_directory=True)

# Visualizar cuántos documentos fueron cargados
logging.info(f"Se cargaron {len(documentos)} documentos exitosamente.")


2024-12-01 23:09:49,252 - INFO - Iniciando carga desde el directorio: data.
2024-12-01 23:09:49,282 - INFO - Archivo 'resumenes_lupus.json' cargado correctamente.
2024-12-01 23:09:49,286 - INFO - Archivo 'resumenes_duchenne.json' cargado correctamente.
2024-12-01 23:09:49,289 - INFO - Archivo 'resumenes_pompe.json' cargado correctamente.
2024-12-01 23:09:49,291 - INFO - Archivo 'resumenes_becker.json' cargado correctamente.
2024-12-01 23:09:49,294 - INFO - Archivo 'resumenes_glycogen storage disease.json' cargado correctamente.
2024-12-01 23:09:49,297 - INFO - Archivo 'resumenes_myotonic dystrophy.json' cargado correctamente.
2024-12-01 23:09:49,297 - INFO - 6 documentos cargados.
2024-12-01 23:09:49,302 - INFO - Se cargaron 6 documentos exitosamente.


### **PASO 4: Configurar la Clave API de Gemini**
Configura la conexión con el modelo de lenguaje Gemini usando la clave API.

In [15]:
gemini_llm = None

def configure_gemini():
    """
    Configura la instancia de Gemini usando la clave API.
    """
    global gemini_llm
    api_key = os.getenv("GEMINI_API_KEY")
    if not api_key:
        logging.error("La clave API de Gemini no está configurada.")
        raise EnvironmentError("Configura GEMINI_API_KEY en tu archivo .env.")
    gemini_llm = Gemini(api_key=api_key)
    logging.info("Gemini configurado correctamente.")

# Configurar Gemini
configure_gemini()

2024-12-01 23:09:50,317 - INFO - Gemini configurado correctamente.


### PASO 5
Configurar el modelo de embeddings.

In [16]:
from sentence_transformers import SentenceTransformer, util
model_name = "all-MiniLM-L6-v2"  # Modelo para embeddings
model = SentenceTransformer(model_name)

def doc_enfermedad(pregunta):
  """
  Identifica el archivo de donde leera la información sobre la enfermedad en la pregunta,
  buscando el mayor embedding entre ella y los nombres de los archivos en documentos
  """
  preg_embedding=model.encode(pregunta)
  archivos = [documentos[i]['filename'] for i in range(0,len(documentos),1)]
  doc_filenames_embeddings = [model.encode(name) for name in archivos]
  similarities = [util.cos_sim(preg_embedding, doc_emb).item() for doc_emb in doc_filenames_embeddings]
  #identificar el índice de la categoría más similar
  max_index = similarities.index(max(similarities))
  max_similarity = similarities[max_index]
  lugar=0
  for i in range(0,len(documentos),1):
    if archivos[i]==max_index:
      lugar=i
  return max_index

2024-12-01 23:09:50,337 - INFO - Use pytorch device_name: mps
2024-12-01 23:09:50,338 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


### PASO 6
Crear clases para documentos e índices.

In [ ]:
# Clase para documentos
class Document:
    def __init__(self, text, metadata=None):
        self.page_content = text
        self.metadata = metadata or {}
    
    def __str__(self):
        # Acceder a los metadatos de forma segura utilizando .get()
        return (
            f"Título: {self.metadata.get('Title', 'N/A')}\n"
            f"Resumen: {self.metadata.get('Summary', 'N/A')}\n"
            f"Tipo de Estudio: {self.metadata.get('StudyType', 'N/A')}\n"
            f"Paises donde se desarrolla el estudio: {self.metadata.get('Countries', 'N/A')}\n"
            f"Fase en que se encuentra el estudio: {self.metadata.get('Phases', 'N/A')}\n"
            f"Identificación en ClinicaTrial: {self.metadata.get('IDestudio', 'N/A')}.\n\n"
        )

# Clase para HNSWlibIndex
class HNSWIndex:
    def __init__(self, embeddings, metadata=None, space='cosine', ef_construction=200, M=16):
        self.dimension = embeddings.shape[1]
        self.index = hnswlib.Index(space=space, dim=self.dimension)
        self.index.init_index(max_elements=embeddings.shape[0], ef_construction=ef_construction, M=M)
        self.index.add_items(embeddings, np.arange(embeddings.shape[0]))
        self.index.set_ef(50)  # Parámetro ef para consultas
        self.metadata = metadata or []
    
    def similarity_search(self, query_vector, k=5):
        labels, distances = self.index.knn_query(query_vector, k=k)
        return [(self.metadata[i], distances[0][j]) for j, i in enumerate(labels[0])]


### PASO 7
Se procesan los documentos y se crea un índice HNSWlib para cada conjunto de documentos.

In [ ]:
# Procesar documentos y crear índice
def desdobla_doc(data2):
    documents = []
    summaries = []

    for entry in data2['content']:
        nctId = entry.get("IDestudio", "")
        briefTitle = entry.get("Title", "")
        summary = entry.get("Summary", "")
        studyType = entry.get("StudyType", "")
        country = entry.get("Countries", "")
        overallStatus = entry.get("OverallStatus", "")
        conditions = entry.get("Conditions", "")
        phases = entry.get("Phasess", "")

        Summary = (
            f"The study titled '{briefTitle}', of type '{studyType}', "
            f"is being conducted to investigate the condition(s) {conditions}. "
            f"This study is briefly summarized as follows: {summary}. "
            f"Currently, the study status is {overallStatus}, and it is taking place in {country}. "
            f"The study is classified under {phases} phase. "
            f"For more information, search {nctId} on ClinicalTrials."
        )
        metadata = {
            "Summary": Summary
        }
        documents.append(Document(Summary, metadata))
        summaries.append(Summary)

    # Crear embeddings y HNSWIndex una sola vez
    embeddings = model.encode([doc.page_content for doc in documents], show_progress_bar=True)
    embeddings = np.array(embeddings).astype(np.float32)  # Asegurarse de que sean float32
    vector_store = HNSWIndex(embeddings, metadata=[doc.metadata for doc in documents])
    return documents, vector_store

# Genera trozos e índices para las enfermedades y los guarda
trozos_archivos = []
index_archivos = []
for i in range(len(documentos)):
    trozos, index = desdobla_doc(documentos[i])
    trozos_archivos.append(trozos)
    index_archivos.append(index)

logging.info("Índices HNSWlib creados para todos los documentos.")

Batches: 100%|██████████| 3/3 [00:02<00:00,  1.13it/s]
2024-12-01 23:12:03,661 - INFO - Índices HNSWlib creados para todos los documentos.


### **PASO 8: Traducir Preguntas y Respuestas**
Traduce preguntas y respuestas entre idiomas según sea necesario utilizando Gemini.

In [39]:
import time
def traducir(texto, idioma_destino):
    """
    Traduce texto al idioma especificado.
    """
    start_time = time.time()  # Medir el tiempo de traducción

    mensajes = [
        ChatMessage(role="system", content="Actúa como un traductor."),
        ChatMessage(role="user", content=f"Por favor, traduce este texto al {idioma_destino}: {texto}")
    ]
    try:
        respuesta = gemini_llm.chat(mensajes)
        elapsed_time = time.time() - start_time  # Tiempo de traducción
        logging.info(f"Traducción completada: {respuesta.message.content.strip()} en {elapsed_time:.2f} segundos.")
        return respuesta.message.content.strip()
    except Exception as e:
        logging.error(f"Error al traducir: {e}")
        return texto  # Devolver texto original como fallback

# Función para generar embedding de una pregunta usando TF-IDF
def generate_embedding(texto):
    """
    Genera un embedding TF-IDF para una pregunta de texto.
    """
    try:
        #embedding = tfidf_vectorizer.transform([texto]).toarray()  # Generar embedding con TF-IDF
        embedding = model.encode([texto])
        logging.info(f"Embedding generado para el texto: {texto}")
        return embedding
    except Exception as e:
        logging.error(f"Error al generar el embedding: {e}")
        return np.zeros((1, 300))  # Devolver un embedding vacío en caso de error

def obtener_contexto(pregunta, index, trozos, top_k=50):
    """
    Recupera los trozos de texto más relevantes para responder la pregunta.
    Traduce la pregunta al inglés antes de buscar en el índice.

    Parámetros:
    - pregunta (str): La pregunta del usuario.
    - index (HNSWIndex): Índice HNSWlib con embeddings.
    - trozos (list): Lista de textos troceados.
    - top_k (int): Número de trozos relevantes a recuperar.

    Retorna:
    - str: Contexto concatenado con los trozos más relevantes.
    """
    try:
        # Traducir la pregunta al inglés
        pregunta_en_ingles = traducir(pregunta, "inglés")
        logging.info(f"Pregunta traducida al inglés: {pregunta_en_ingles}")

        # Generar embedding de la pregunta traducida
        pregunta_emb = generate_embedding(pregunta_en_ingles)
        logging.info("Embedding generado para la pregunta.")

        # Usar indice correspondiente a la enfermedad sobre la que se pregunta
        results = index.similarity_search(pregunta_emb, k=top_k)
        texto = ""
        for entry in results:
            resum = entry[0]["Summary"]
            texto += resum + "\n"

        # Realizar búsqueda en el índice HNSWlib
        #labels, _ = index.knn_query(pregunta_emb.reshape(1, -1), k=top_k)
        #contexto = ' '.join(trozos[i] for i in labels[0])
        contexto = texto
        logging.info("Contexto relevante recuperado para la pregunta.")
        return contexto
    except Exception as e:
        logging.error(f"Error al obtener el contexto: {e}")
        return ""

### **PASO 9: Generar Respuestas**
Genera respuestas utilizando el modelo Gemini y el contexto proporcionado.

In [40]:
# Mejorar la Generación de Respuestas con Prompts Específicos
def categorizar_pregunta(pregunta):
    """
    Clasifica la pregunta en categorías específicas como 'tratamiento', 'ensayo clínico', etc.
    """
    categorias = {
        "tratamiento": ["tratamiento", "medicación", "cura", "terapia", "fármaco"],
        "ensayo": ["ensayo", "estudio", "prueba", "investigación", "trial"],
        "resultado": ["resultado", "efectividad", "resultados", "éxito", "fracaso"],
        "prevención": ["prevención", "previene", "evitar", "reducción de riesgo"]
    }
    for categoria, palabras in categorias.items():
        if any(palabra in pregunta.lower() for palabra in palabras):
            return categoria
    return "general"

def generar_prompt(categoria, pregunta):
    """
    Genera un prompt específico basado en la categoría de la pregunta.
    """
    prompts = {
        "tratamiento": f"Proporciona información detallada sobre tratamientos relacionados con: {pregunta}.",
        "ensayo": f"Describe los ensayos clínicos actuales relacionados con: {pregunta}.",
        "resultado": f"Explica los resultados más recientes de ensayos clínicos sobre: {pregunta}.",
        "prevención": f"Ofrece estrategias de prevención para: {pregunta}."
    }
    return prompts.get(categoria, "Por favor, responde la pregunta sobre ensayos clínicos.")


# Se define una función para detectar si una pregunta es un saludo.
def es_saludo(pregunta):
    saludos = ["hola", "buen día", "buenas", "cómo estás", "cómo te llamas", "qué tal", "estás bien", "buenas tardes", "buenas noches"]
    return any(saludo in pregunta.lower() for saludo in saludos)

def responder_saludo():
    saludos_respuestas = [
        "¡Hola! Estoy para ayudarte con información sobre ensayos clínicos. ¿En qué puedo asistirte hoy?",
        "¡Buenas! Tenés alguna pregunta sobre ensayos clínicos en enfermedades neuromusculares?",
        "¡Hola! ¿Cómo puedo ayudarte con tus consultas sobre ensayos clínicos?"
    ]
    import random
    return random.choice(saludos_respuestas)

def generar_respuesta(pregunta, contexto, prompt_especifico):
    """
    Genera una respuesta usando el contexto proporcionado y un prompt específico.
    """
    start_time = time.time()  # Medir el tiempo de generación de respuesta

    mensajes = [
        ChatMessage(role="system", content="Eres un experto médico."),
        ChatMessage(role="user", content=f"{prompt_especifico}\nContexto: {contexto}\nPregunta: {pregunta}")
    ]
    try:
        respuesta = gemini_llm.chat(mensajes)
        elapsed_time = time.time() - start_time  # Tiempo de generación de respuesta
        logging.info(f"Respuesta generada en inglés: {respuesta.message.content.strip()} en {elapsed_time:.2f} segundos.")

        # Traducir la respuesta al español
        respuesta_en_espanol = traducir(respuesta.message.content, "español")
        logging.info(f"Respuesta traducida al español: {respuesta_en_espanol}")
        return respuesta_en_espanol
    except Exception as e:
        logging.error(f"Error al generar la respuesta: {e}")
        return "Lo siento, ocurrió un error al generar la respuesta."


### **PASO 10: Función Principal para Responder Preguntas**
Integra todos los pasos previos para traducir, recuperar contexto y generar respuestas.

In [41]:
import json
import os
import hashlib

def generar_hash(pregunta):
    """
    Genera un hash SHA-256 para una pregunta dada.
    """
    return hashlib.sha256(pregunta.encode('utf-8')).hexdigest()

def obtener_respuesta_cacheada(pregunta):
    """
    Obtiene la respuesta de un archivo de caché si ya existe.
    """
    hash_pregunta = generar_hash(pregunta)
    archivo_cache = f"cache/{hash_pregunta}.json"
    if os.path.exists(archivo_cache):
        try:
            with open(archivo_cache, "r", encoding='utf-8') as f:
                datos = json.load(f)
                return datos.get("respuesta", None)
        except Exception as e:
            logging.error(f"Error al leer el caché para la pregunta '{pregunta}': {e}")
            return None
    return None

def guardar_respuesta_cacheada(pregunta, respuesta):
    """
    Guarda la respuesta en caché para futuras consultas.
    """
    hash_pregunta = generar_hash(pregunta)
    archivo_cache = f"cache/{hash_pregunta}.json"
    try:
        os.makedirs(os.path.dirname(archivo_cache), exist_ok=True)  # Asegura que el directorio exista
        with open(archivo_cache, "w", encoding='utf-8') as f:
            json.dump({"pregunta": pregunta, "respuesta": respuesta}, f, ensure_ascii=False, indent=4)
        logging.info(f"Respuesta cacheada para la pregunta: '{pregunta}'")
    except Exception as e:
        logging.error(f"Error al guardar la respuesta en caché para la pregunta '{pregunta}': {e}")
        

def responder_pregunta(pregunta, index, trozos):
    """
    Integra todos los pasos: categorización, traducción, recuperación de contexto y generación de respuestas.
    Incluye manejo de caché para optimizar la velocidad de respuesta.
    """
    try:
        # Verificar si la respuesta ya está en el caché
        respuesta_cacheada = obtener_respuesta_cacheada(pregunta)
        if respuesta_cacheada:
            logging.info(f"Respuesta obtenida del caché para la pregunta: '{pregunta}'")
            return respuesta_cacheada
        
        # Categorizar la pregunta
        categoria = categorizar_pregunta(pregunta)
        logging.info(f"Categoría de la pregunta: {categoria}")

        # Generar prompt específico basado en la categoría
        prompt_especifico = generar_prompt(categoria, pregunta)  # Añadir 'pregunta' aquí
        logging.info(f"Prompt específico generado: {prompt_especifico}")

        # Obtener el contexto relevante para la pregunta
        contexto = obtener_contexto(pregunta, index, trozos)
        if not contexto:
            logging.warning("No se encontró un contexto relevante para la pregunta.")
            respuesta = "No pude encontrar información relevante para responder tu pregunta."
            guardar_respuesta_cacheada(pregunta, respuesta)
            return respuesta

        # Generar la respuesta utilizando el contexto y el prompt específico
        respuesta = generar_respuesta(pregunta, contexto, prompt_especifico)
        
        # Guardar la respuesta en el caché
        guardar_respuesta_cacheada(pregunta, respuesta)
        
        return respuesta
    except Exception as e:
        logging.error(f"Error en el proceso de responder pregunta: {e}")
        return "Ocurrió un error al procesar tu pregunta."





### **PASO 11: Interfaz CLI**
Proporciona una interfaz interactiva para que los usuarios puedan hacer preguntas.

In [42]:
if __name__ == "__main__":
    # Asegurar que el directorio de caché exista
    os.makedirs("cache", exist_ok=True)
    
    print("Bienvenido al chatbot de ensayos clínicos")
    print("Conversemos sobre Ensayos Clínicos\n de las siguientes enfermedades neuromusculares: Distrofia Muscular de Duchenne o de Becker, Enfermedad de Pompe, Distrofia Miotónica o Enfermedad de almacenamiento de glucosis")
    print("Escribe tu pregunta, dejando claramente expresada la enfermedad sobre la que quieres conocer información de ensayos médicos. Escribe 'salir' para terminar.")
    while True:
        # Solicitar la pregunta al usuario
        pregunta = input("Tu pregunta: ").strip()

        # Verificar si el usuario desea salir
        if pregunta.lower() in ['salir', 'chau', 'exit', 'quit']:
            print("¡Chau!")
            logging.info("El usuario ha finalizado la sesión.")
            break

        # Verificar si la pregunta es un saludo
        if es_saludo(pregunta):
            respuesta_saludo = responder_saludo()
            print(respuesta_saludo)
            logging.info("Se detectó un saludo del usuario.")
            continue

        # Identificar la enfermedad relacionada con la pregunta
        idn = doc_enfermedad(pregunta)
        index = index_archivos[idn]
        trozos = trozos_archivos[idn]

        # Generar la respuesta para la pregunta del usuario
        respuesta = responder_pregunta(pregunta, index, trozos)
        print(f"Respuesta: {respuesta}")


Bienvenido al chatbot de ensayos clínicos
Conversemos sobre Ensayos Clínicos
 de las siguientes enfermedades neuromusculares: Distrofia Muscular de Duchenne o de Becker, Enfermedad de Pompe, Distrofia Miotónica o Enfermedad de almacenamiento de glucosis
Escribe tu pregunta, dejando claramente expresada la enfermedad sobre la que quieres conocer información de ensayos médicos. Escribe 'salir' para terminar.


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.19it/s]
2024-12-01 23:41:38,713 - INFO - Categoría de la pregunta: ensayo
2024-12-01 23:41:38,713 - INFO - Prompt específico generado: Describe los ensayos clínicos actuales relacionados con: que enfermedades neuromusculares se están tratando en ensayos en Argentina?.
2024-12-01 23:41:40,035 - INFO - Traducción completada: What neuromuscular diseases are being treated in clinical trials in Argentina? en 1.32 segundos.
2024-12-01 23:41:40,035 - INFO - Pregunta traducida al inglés: What neuromuscular diseases are being treated in clinical trials in Argentina?
Batches: 100%|██████████| 1/1 [00:00<00:00,  2.47it/s]
2024-12-01 23:41:40,446 - INFO - Embedding generado para el texto: What neuromuscular diseases are being treated in clinical trials in Argentina?
2024-12-01 23:41:40,447 - INFO - Embedding generado para la pregunta.
2024-12-01 23:41:40,448 - INFO - Contexto relevante recuperado para la pregunta.
2024-12-01 23:41:41,801 - INFO - Resp

Respuesta: Según la información proporcionada, no hay ensayos clínicos en curso en Argentina para enfermedades neuromusculares registrados en la base de datos ClinicalTrials.gov que me has facilitado. Todos los estudios mencionados se realizan en otros países. Para obtener información sobre ensayos clínicos en Argentina para enfermedades neuromusculares, es necesario consultar bases de datos específicas de ensayos clínicos argentinos o contactar directamente con centros de investigación médica en Argentina.


Batches: 100%|██████████| 1/1 [00:00<00:00, 24.69it/s]
2024-12-01 23:42:17,071 - INFO - Categoría de la pregunta: ensayo
2024-12-01 23:42:17,071 - INFO - Prompt específico generado: Describe los ensayos clínicos actuales relacionados con: que ensayos clinicos se están estudiando en Argentina?.
2024-12-01 23:42:17,523 - INFO - Traducción completada: What clinical trials are being studied in Argentina? en 0.45 segundos.
2024-12-01 23:42:17,524 - INFO - Pregunta traducida al inglés: What clinical trials are being studied in Argentina?
Batches: 100%|██████████| 1/1 [00:00<00:00, 25.44it/s]
2024-12-01 23:42:17,568 - INFO - Embedding generado para el texto: What clinical trials are being studied in Argentina?
2024-12-01 23:42:17,569 - INFO - Embedding generado para la pregunta.
2024-12-01 23:42:17,571 - INFO - Contexto relevante recuperado para la pregunta.
2024-12-01 23:42:19,035 - INFO - Respuesta generada en inglés: Basado en la información proporcionada, no hay ensayos clínicos sobre dis

Respuesta: Según la información proporcionada, no existen ensayos clínicos sobre distrofia miotónica u otras enfermedades mencionadas que se estén realizando en Argentina. La lista de ensayos clínicos incluye estudios en diversos países, pero Argentina no aparece en ninguno. Para encontrar ensayos clínicos en Argentina, sería necesaria una búsqueda más exhaustiva en bases de datos de ensayos clínicos, especificando "Argentina" como ubicación.


...

Respuesta: La información proporcionada describe numerosos estudios clínicos relacionados con la Distrofia Muscular de Duchenne (DMD). Estos estudios abarcan una variedad de enfoques, incluyendo:

* **Estudios observacionales:** Estos estudios, como "Estudio Longitudinal de la Historia Natural de la Distrofia Muscular de Duchenne (DMD)" y otros similares, se centran en observar la progresión natural de la enfermedad sin intervención terapéutica. Su objetivo es comprender mejor la historia natural de la DMD, identificar biomarcadores y establecer medidas de resultado para futuros ensayos clínicos. Algunos estudios observacionales se centran en aspectos específicos, como la evaluación de la extremidad superior, la función cardiopulmonar o la evaluación de las necesidades neurodesarrolladoras.

* **Estudios intervencionales:** Estos estudios, como "Encontrando el Régimen Óptimo para la Distrofia Muscular de Duchenne" y muchos otros, prueban intervenciones terapéuticas, incluyendo diferent

Batches: 100%|██████████| 1/1 [00:00<00:00, 27.04it/s]
2024-12-01 23:43:56,546 - INFO - Categoría de la pregunta: general
2024-12-01 23:43:56,546 - INFO - Prompt específico generado: Por favor, responde la pregunta sobre ensayos clínicos.
2024-12-01 23:43:57,762 - INFO - Traducción completada: The most accurate translation depends on the context.  Here are a few options:

* **What is Duchenne?** (This is the most straightforward and likely the best translation if you're asking about Duchenne muscular dystrophy.)
* **What's Duchenne?** (A more informal version of the above.)
* **What is Duchenne's?** (This is acceptable, but "Duchenne" is generally preferred as a standalone term.)


If the context is different, please provide more information so I can give a more precise translation. en 1.21 segundos.
2024-12-01 23:43:57,762 - INFO - Pregunta traducida al inglés: The most accurate translation depends on the context.  Here are a few options:

* **What is Duchenne?** (This is the most str

Respuesta: La distrofia muscular de Duchenne (DMD) es una enfermedad genética hereditaria que afecta principalmente a los varones.  Está causada por una mutación en el gen que codifica la distrofina, una proteína esencial para la estructura y la función de los músculos. La ausencia o deficiencia de distrofina provoca una degeneración muscular progresiva, debilidad y pérdida de la función motora.

La DMD es una enfermedad progresiva, lo que significa que los síntomas empeoran con el tiempo. Los primeros síntomas suelen aparecer en la infancia temprana, entre los 2 y 5 años de edad, y pueden incluir debilidad muscular, dificultad para caminar, caídas frecuentes y retraso en el desarrollo motor. A medida que la enfermedad progresa, la debilidad muscular se extiende a otros grupos musculares, afectando la capacidad para realizar actividades cotidianas, como subir escaleras o levantarse del suelo.  Eventualmente, la DMD puede afectar los músculos respiratorios y cardíacos, lo que puede llev

2024-12-01 23:44:19,729 - INFO - El usuario ha finalizado la sesión.


¡Chau!
